<a href="https://colab.research.google.com/github/gg2001/transformer-circuits/blob/master/puzzles/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Mech Interp Puzzle 1: Suspiciously Similar Embeddings in GPT-Neo](https://www.alignmentforum.org/posts/eLNo7b56kQQerCzp2/mech-interp-puzzle-1-suspiciously-similar-embeddings-in-gpt)

In [1]:
%pip install transformer_lens plotly nbformat

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62

In [2]:
from tqdm import tqdm
from transformer_lens import HookedTransformer
import plotly.express as px
import torch
import heapq

In [4]:
model = HookedTransformer.from_pretrained("gpt-neo-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Loaded pretrained model gpt-neo-small into HookedTransformer


In [5]:
W_E = model.W_E
W_E.requires_grad = False

In [6]:
# subsample = torch.randperm(model.cfg.d_vocab)[:5000].to(model.cfg.device)
# W_E = model.W_E[subsample]  # Take a random subset of 5,000 for memory reasons

In [7]:
W_E_normed = W_E / W_E.norm(dim=-1, keepdim=True) # [d_vocab, d_model]
# cosine_sims = W_E_normed @ W_E_normed.T # [d_vocab, d_vocab]

In [73]:
num_tokens = W_E_normed.size(0)
batch = 1000
top_k = 1000
heap = []

for i in tqdm(range(0, num_tokens, batch)):
    token_cosine_sims = (W_E_normed[i:i+batch] @ W_E_normed[i + 1:].T)
    mask = torch.tril(torch.ones(token_cosine_sims.size(0), token_cosine_sims.size(1), dtype=torch.bool, device=token_cosine_sims.device), diagonal=-1)
    cosine_sims_top = token_cosine_sims.masked_fill(mask, float('-inf'))
    top_cosine_sims = torch.topk(cosine_sims_top.view(-1), top_k)
    top_token1 = top_cosine_sims.indices // cosine_sims.size(1)
    top_token2 = top_cosine_sims.indices % cosine_sims.size(1)
    for token_0, token_1, cosine_sim in zip(top_token1.tolist(), top_token2.tolist(), top_cosine_sims.values.tolist()):
        if len(heap) < top_k:
            heapq.heappush(heap, (cosine_sim, token_0, token_1))
        else:
            if cosine_sim > heap[0][0]:
                heapq.heappushpop(heap, (cosine_sim, token_0, token_1))

100%|██████████| 51/51 [00:00<00:00, 196.17it/s]


In [74]:
for cosine_sim, token_0, token_1 in heap:
    print(f"Token 0: {model.to_string([int(token_0)])}, Token 1: {model.to_string([int(token_1)])}, Cosine Similarity: {cosine_sim}")

Token 0: R, Token 1:  adventure, Cosine Similarity: 0.9999060034751892
Token 0: u, Token 1:  Dolphins, Cosine Similarity: 0.9999061226844788
Token 0: �, Token 1:  layoffs, Cosine Similarity: 0.9999060034751892
Token 0: R, Token 1:  Adobe, Cosine Similarity: 0.9999062418937683
Token 0: ke, Token 1: Merit, Cosine Similarity: 0.9999063611030579
Token 0: /, Token 1:  Loki, Cosine Similarity: 0.9999063014984131
Token 0: �, Token 1:  trailed, Cosine Similarity: 0.9999060034751892
Token 0: �, Token 1:  Fnatic, Cosine Similarity: 0.9999063014984131
Token 0: �, Token 1:  indign, Cosine Similarity: 0.9999068975448608
Token 0: u, Token 1:  Codex, Cosine Similarity: 0.9999064207077026
Token 0: ion, Token 1:  Palestinian, Cosine Similarity: 0.9999067783355713
Token 0: �, Token 1:  score, Cosine Similarity: 0.9999065399169922
Token 0: �, Token 1:  appendix, Cosine Similarity: 0.9999063611030579
Token 0: ion, Token 1:  Actually, Cosine Similarity: 0.9999062418937683
Token 0: �, Token 1: Optional, Cos

In [ ]:
# px.histogram(
#    cosine_sims.flatten().detach().cpu().numpy(),
#    title="Pairwise cosine sims of embedding",
#)